In [147]:
import pandas as pd
import numpy as np
from scipy.special import softmax,  expit
from scipy.stats import zscore
import ast
import os
import os.path as osp

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
from miga.const import ID2LABELS_SMG_SHORT as ID2LABELS

In [148]:
LABELS2ID = {v:k for k,v in ID2LABELS.items()}

In [149]:
path_to_folder = r'D:\Project-mpg microgesture\human_micro_gesture_classifier\prediction_over_videos'
# file_name = 'prediction_over_videos_Sample0031_color_935_2000.csv'
# file_name = 'prediction_over_videos_Sample0031_color_12400_13700.csv'
file_name = 'prediction_over_videos_Sample0031_color_21800_22200.csv'
path_to_df = osp.join(path_to_folder, file_name)


# start_frame = 12400
# sample_ratio = 2
start_frame = 21800
sample_ratio = 8




df = pd.read_csv(path_to_df, header=None, index_col=None)
df.columns  = ['frame','resolution','logits','smoothed_logits','predictions','labels']

df['logits_np'] = df['logits'].apply(lambda x: np.array(ast.literal_eval(x.replace('\n','').replace(' ',',').replace(',,',',').replace(',,',',').replace(',,',','))))
# df['sigmoid'] = df['logits_np'].apply(lambda x: expit(x))
df['sigmoid'] = df['logits_np'].apply(lambda x: zscore(x))
# df['sigmoid'] = df['logits_np']




df['true_frame'] = df['frame'].apply(lambda x: x * sample_ratio + start_frame)

labels_columns = ID2LABELS.values()
df_split = pd.DataFrame(df['sigmoid'].tolist(), columns=labels_columns)
df_split['frame'] = df['true_frame']
df_split['resolution'] = df['resolution']

In [150]:
df_split.columns

Index(['Turtling neck', 'Rubbing face', 'Folding arms', 'Touching nose',
       'Moving legs', 'Scratching neck', 'Arms behind', 'Rubbing hands',
       'Arms akimbo', 'Crossing legs', 'Scratching body', 'Touching face',
       'Adjusting hair', 'Holding arms', 'Pulling collar', 'Playing wobjects',
       'Hand gestures', 'frame', 'resolution'],
      dtype='object')

In [151]:
# Assuming df is your DataFrame

# Step 1: Group by "frame"
grouped = df_split.copy().groupby('frame')
ret = []
# Step 2 and 3: Calculate mean and update rows
for frame, group in grouped:
    # Calculate mean excluding rows with resolution as "aggregated"
    mean_values = group.loc[group['resolution'] != 'aggregated'][labels_columns].mean().values.reshape(1,-1)
    
    # Update rows where resolution is "aggregated" with the mean values
    aggregated_mask = (group['resolution'] == 'aggregated')
    group.loc[aggregated_mask,labels_columns] = mean_values  # Replace 'value_column' with the actual column name you want to update
    ret.append(group)
df_split_mean = pd.concat(ret)

In [152]:
df_split_mean.resolution.unique()

array(['aggregated', '16', '32', '64'], dtype=object)

In [153]:
def plot_pred_over_time(df, label=0, res='32'):
    df = df.loc[df['resolution'] == res]
    vals = np.stack(df['sigmoid'].values)[:, label]
    frames = df['frame'].values * sample_ratio + start_frame
    fig, ax = plt.subplots(figsize=(18, 10))
    ax.plot(frames, vals, label=label)
    ax.legend()

    

In [154]:
# def plot_pred_over_time(df, label=0, res='32'):
#     # Filter the DataFrame based on the given resolution
#     df = df.loc[df['resolution'] == res]
# 
#     # Extract the values for the specified label
#     vals = np.stack(df['sigmoind'].values)
#     frames = df['frame'].values * sample_ratio + start_frame
# 
#     fig = go.Figure()
# 
#     # Add the traces for each label
#     for i in range(vals.shape[1]):
#         fig.add_trace(go.Scatter(
#             x=frames,
#             y=vals[:, i],
#             mode='lines+markers',
#             name=f'Label {i}',
#             text=[f'Label: {i}, Value: {val:.2f} Frame:{frames}' for val in vals[:, i]],
#             hoverinfo='text+y',
#             visible=True
#         ))
# 
#     # Create a dropdown menu with tick boxes for each label
#     visibility = [True] * vals.shape[1]
# 
#     buttons = []
#     for i in range(vals.shape[1]):
#         visibility_list = [False] * vals.shape[1]
#         visibility_list[i] = True
#         buttons.append(
#             dict(
#                 args=[{"visible": visibility_list}],
#                 label=f"Label {i}",
#                 method="update"
#             )
#         )
# 
#     # Adding an 'All' button to make all traces visible
#     buttons.insert(0, dict(
#         args=[{"visible": [True] * vals.shape[1]}],
#         label="All",
#         method="update"
#     ))
# 
#     fig.update_layout(
#         title='Predicted Values Over Time',
#         xaxis_title='Frame',
#         yaxis_title='Values',
#         legend_title='Labels',
#         hovermode='closest',
#         updatemenus=[
#             dict(
#                 type='dropdown',
#                 showactive=True,
#                 buttons=buttons,
#                 direction='down',
#                 pad={"r": 10, "t": 10},
#                 x=0.1,
#                 xanchor='left',
#                 y=1.15,
#                 yanchor='top'
#             ),
#         ]
#     )
# 
#     return fig

In [155]:
# def plot_pred_over_time_plotly(df, res='32'):
#     df_filtered = df[df['resolution'] == res]
#     # 
#     # frames = df_filtered['frame'].values * sample_ratio + start_frame
#     # 
#     # fig = go.Figure()
#     fig = px.line(df_filtered, x='frame', y=)
#     # Show plot 
#     fig.show()
#     # fig.add_trace(px.scatter(df, x='true_frame', y='sigmoid'))
# 
#     # fig.update_layout(title='Predictions Over Time',
#     #                   xaxis_title='Frame Number',
#     #                   yaxis_title='Value')
#     # 
#     # fig.update_traces(hoverinfo='text',
#     #                   hovertext=[f'Label: {label}<br>Value: {vals[i]}<br>Frame: {frames[i]}' for i in range(len(frames))])
# 
#     fig.show()

In [156]:
def plot_traces(df_split, resolution='64'):
# resolution='64'
        # Filter the relevant row corresponding to the input "resolution"
    df_filtered = df_split[df_split['resolution'] == resolution].drop(columns=['resolution'])
    
    # Extract the values from the DataFrame
    # Extract the values from the DataFrame
    frames = df_filtered['frame']
    labels = df_filtered.loc[:, labels_columns]  # Select columns from 0 to 16
    
    
    # Create the Plotly figure
    fig = go.Figure()
    
    # Add each trace to the plot
    for i, label in enumerate(labels_columns):
        fig.add_trace(go.Scatter(x=frames, y=labels[label], mode='lines', name=f'{label}:{i+1}',
                      hovertemplate=[f'Frame: {frames.values[k]}<br>Pred:{labels[label].values[k]:.3f}' for k in range(len(frames))]))
    fig.update_traces(mode="lines+text")
    # Update layout
    fig.update_layout(title=f'Predictions Over Time:{resolution}',
                      xaxis_title='Frame Number',
                      yaxis_title='Value',
                       autosize=False,
                        width=1200,
                        height=600)
    # 
    # ret = []
    # for label in labels_columns:
    #     label_txt = []
    #     for i in range(len(frames)):
    #         txt = f'Label: {label}:{LABELS2ID[label]}<br>Value: {labels[label].values[i]:0.3f}<br>Frame: {frames.values[i]}'
    #         label_txt.append(txt)
    #     ret.append(label_txt)
    # # Update hover text to display the full frame number
    # # print(frames)
    # fig.update_traces(hoverinfo='text',
    #                   hovertext=ret)   # 
    #  # Show the plot
    fig.show()

In [157]:
plot_traces(df_split, resolution='aggregated')
plot_traces(df_split, resolution='16')
plot_traces(df_split, resolution='32')
plot_traces(df_split, resolution='64')